In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy.interpolate import interp2d
import gsw
import os

In [4]:
state0 = xr.open_dataset('/Users/weiyangbao/Desktop/sill12/state_002.nc')
state = state0.isel(Y = range(1,10)) # Remove the boundary walls in y direction

In [7]:
xloc = 28 # Location in x-axsis 0~50
t = 41 # Time
pt = state.Temp.isel(T = t)
s = state.S.isel(T = t)
u = state.U.isel(T = t)
depth = state.Z.data
# Mean value in Y direction, boundaries removed
pt_ymean = pt.data.mean(1)
s_ymean = s.data.mean(1)
u_ymean = (u.data[:, :, 1:].mean(1) + u.data[:, :, :-1].mean(1)) / 2


(50, 50)